In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict, Counter
import xgboost as xgb
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss, mean_squared_log_error
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD, PCA
import tqdm
import sys
import os
import gc
import argparse
import warnings
warnings.filterwarnings('ignore')

In [2]:
trans_info = pd.read_csv('./data/账户交易信息.csv')
static_info = pd.read_csv('./data/账户静态信息.csv')

train_label = pd.read_csv('./data/训练集标签.csv')
test_label = pd.read_csv('./data/test_dataset.csv')

In [3]:
def get_time_feature(df, col):
    
    df_copy = df.copy()
    prefix = col + "_"
    df_copy['new_'+col] = df_copy[col].astype(str)
    
    col = 'new_'+col
    df_copy[col] = pd.to_datetime(df_copy[col], format='%Y-%m-%d')
    df_copy[prefix + 'month'] = df_copy[col].dt.month
    df_copy[prefix + 'day'] = df_copy[col].dt.day
    df_copy[prefix + 'weekofyear'] = df_copy[col].dt.weekofyear
    df_copy[prefix + 'dayofyear'] = df_copy[col].dt.dayofyear
    df_copy[prefix + 'dayofweek'] = df_copy[col].dt.dayofweek
    df_copy[prefix + 'is_wknd'] = df_copy[col].dt.dayofweek // 6
    df_copy[prefix + 'is_month_start'] = df_copy[col].dt.is_month_start.astype(int)
    df_copy[prefix + 'is_month_end'] = df_copy[col].dt.is_month_end.astype(int)
    del df_copy[col]
    
    df_copy[prefix + 'hour'] = df_copy['jysj'].apply(lambda x:int(x.split(':')[0]))
    df_copy[prefix + 'minu'] = df_copy['jysj'].apply(lambda x:int(x.split(':')[1]))
    df_copy[prefix + 'date'] = df_copy['jysj'].apply(lambda x:int(x.split(':')[0])*60 + int(x.split(':')[1]))
    
    return df_copy   

trans_info = get_time_feature(trans_info, "jyrq")
time_cols = [f for f in trans_info.columns if 'jyrq_' in f]
print(time_cols)
print(train_label.shape, test_label.shape)

['jyrq_month', 'jyrq_day', 'jyrq_weekofyear', 'jyrq_dayofyear', 'jyrq_dayofweek', 'jyrq_is_wknd', 'jyrq_is_month_start', 'jyrq_is_month_end', 'jyrq_hour', 'jyrq_minu', 'jyrq_date']
(1200, 2) (4800, 1)


In [4]:
def get_base_feat(df1_, df2_):
    df1 = df1_.copy() # 构建特征数据 
    df2 = df2_.copy()

    agg_func = {
        'dfzh': ['nunique','count'],
        'dfhh': ['nunique'],
        'jyqd': ['nunique'],
        'zydh': ['nunique'],
        'jyje': ['sum','mean','max','min','std',np.ptp],
        'zhye': ['sum','mean','max','min','std',np.ptp],
        'dfmccd': ['mean','max','min','std',np.ptp],
    }
    
    for col in time_cols:
        agg_func[col] = ['mean','min','max',np.ptp]
    
    agg_df = df1[df1['jdbj']==0].groupby(['zhdh']).agg(agg_func).reset_index()
    agg_df.columns = ['zhdh'] + ['zhdh_jdbj0_' + '_'.join(f).strip() for f in agg_df.columns.values if f[0] not in ['zhdh']]
    df2 = df2.merge(agg_df, on=['zhdh'], how='left')
    
    agg_df = df1[df1['jdbj']==1].groupby(['zhdh']).agg(agg_func).reset_index()
    agg_df.columns = ['zhdh'] + ['zhdh_jdbj1_' + '_'.join(f).strip() for f in agg_df.columns.values if f[0] not in ['zhdh']]
    df2 = df2.merge(agg_df, on=['zhdh'], how='left')
    
    return df2

train_label = get_base_feat(trans_info, train_label)
test_label = get_base_feat(trans_info, test_label)

In [5]:
static_info['khrq']  = pd.to_datetime(static_info['khrq'], format='%Y-%m-%d')
static_info['year']  = static_info['khrq'].dt.year
static_info['month'] = static_info['khrq'].dt.month
static_info['day']   = static_info['khrq'].dt.day

# 自然数编码
def label_encode(series):
    unique = list(series.unique())
    return series.map(dict(zip(
        unique, range(series.nunique())
    )))

for col in ['khjgdh']:
    static_info[col] = label_encode(static_info[col])

keep_cols = ['zhdh','year','month','day','khjgdh','xb','年龄']

train_label = train_label.merge(static_info[keep_cols], on=['zhdh'], how='left')
test_label  = test_label.merge(static_info[keep_cols], on=['zhdh'], how='left')

In [6]:
cols = [f for f in train_label.columns if f not in ['zhdh','black_flag']]
len(cols)

138

In [7]:
def cv_model(clf, train_x, train_y, test_x, clf_name):
    folds = 5
    seed = 2023
    kf = KFold(n_splits=folds, shuffle=True, random_state=seed)
    
    oof = np.zeros(train_x.shape[0])
    predict = np.zeros(test_x.shape[0])

    cv_scores = []

    for i, (train_index, valid_index) in enumerate(kf.split(train_x, train_y)):
        print('************************************ {} ************************************'.format(str(i+1)))
        trn_x, trn_y, val_x, val_y = train_x.iloc[train_index], train_y[train_index], train_x.iloc[valid_index], train_y[valid_index]

        if clf_name == "lgb":
            train_matrix = clf.Dataset(trn_x, label=trn_y)
            valid_matrix = clf.Dataset(val_x, label=val_y)

            params = {
                'boosting_type': 'gbdt',
                'objective': 'binary',
                'metric': 'auc',
                'min_child_weight': 5,
                'num_leaves': 2 ** 5,
                'lambda_l2': 10,
                'feature_fraction': 0.8,
                'bagging_fraction': 0.8,
                'bagging_freq': 4,
                'learning_rate': 0.01,
                'seed': 2020,
                'n_jobs':8
            }

            model = clf.train(params, train_matrix, 10000, valid_sets=[train_matrix, valid_matrix], 
                              categorical_feature=[], verbose_eval=200, early_stopping_rounds=200)
            val_pred = model.predict(val_x, num_iteration=model.best_iteration)
            test_pred = model.predict(test_x, num_iteration=model.best_iteration)
            
            print(list(sorted(zip(features, model.feature_importance("gain")), key=lambda x: x[1], reverse=True))[:20])
                
        if clf_name == "xgb":
            train_matrix = clf.DMatrix(trn_x , label=trn_y)
            valid_matrix = clf.DMatrix(val_x , label=val_y)
            test_matrix = clf.DMatrix(test_x)
            
            params = {'booster': 'gbtree',
                      'objective': 'binary:logistic',
                      'eval_metric': 'auc',
                      'gamma': 1,
                      'min_child_weight': 1.5,
                      'max_depth': 5,
                      'lambda': 10,
                      'subsample': 0.7,
                      'colsample_bytree': 0.7,
                      'colsample_bylevel': 0.7,
                      'eta': 0.05,
                      'tree_method': 'exact',
                      'seed': 2020,
                      'nthread': 8
                      }
            
            watchlist = [(train_matrix, 'train'),(valid_matrix, 'eval')]
            
            model = clf.train(params, train_matrix, num_boost_round=10000, evals=watchlist, verbose_eval=1000, early_stopping_rounds=500)
            val_pred  = model.predict(valid_matrix, ntree_limit=model.best_ntree_limit)
            test_pred = model.predict(test_matrix , ntree_limit=model.best_ntree_limit)
                 
        if clf_name == "cat":
            
            model = clf(
                        n_estimators=10000,
                        random_seed=1024,
                        eval_metric='AUC',
                        learning_rate=0.05,
                        max_depth=5,
                        early_stopping_rounds=200,
                        metric_period=500,
                    )

            model.fit(trn_x, trn_y, eval_set=(val_x, val_y),
                      use_best_model=True,
                      verbose=1)
            
            val_pred  = model.predict_proba(val_x)[:,1]
            test_pred = model.predict_proba(test_x)[:,1]
            
        oof[valid_index] = val_pred
        predict += test_pred / kf.n_splits
        
        cv_scores.append(roc_auc_score(val_y, val_pred))
        print(cv_scores)
       
    return oof, predict

In [8]:
xgb_oof, xgb_pred = cv_model(xgb, train_label[cols], train_label['black_flag'], test_label[cols], 'xgb')

************************************ 1 ************************************
[0]	train-auc:0.91325	eval-auc:0.88884
[546]	train-auc:0.99957	eval-auc:0.95981
[0.9683333333333333]
************************************ 2 ************************************
[0]	train-auc:0.93143	eval-auc:0.89543
[989]	train-auc:0.99975	eval-auc:0.95651
[0.9683333333333333, 0.9586806814063067]
************************************ 3 ************************************
[0]	train-auc:0.93816	eval-auc:0.87224
[645]	train-auc:0.99968	eval-auc:0.94899
[0.9683333333333333, 0.9586806814063067, 0.9507616707616707]
************************************ 4 ************************************
[0]	train-auc:0.93303	eval-auc:0.91848
[607]	train-auc:0.99964	eval-auc:0.96099
[0.9683333333333333, 0.9586806814063067, 0.9507616707616707, 0.9695094610348848]
************************************ 5 ************************************
[0]	train-auc:0.94041	eval-auc:0.90042
[642]	train-auc:0.99973	eval-auc:0.94667
[0.9683333333333

In [9]:
oof = xgb_oof
scores = []; thresholds = []
best_score = 0; best_threshold = 0

for threshold in np.arange(0.4,0.6,0.01):
    preds = (oof.reshape((-1))>threshold).astype('int')
    m = f1_score(train_label['black_flag'].values.reshape((-1)), preds, average='macro')   
    scores.append(m)
    thresholds.append(threshold)
    if m>best_score:
        best_score = m
        best_threshold = threshold
    print(f'{threshold:.02f}, {m}')
print(f'{best_threshold:.02f}, {best_score}')

0.40, 0.9125694642230273
0.41, 0.9111209021608255
0.42, 0.9098748800946124
0.43, 0.9106975892165431
0.44, 0.9092266165178833
0.45, 0.907966637906241
0.46, 0.9064765022211831
0.47, 0.9049738098549112
0.48, 0.9049738098549112
0.49, 0.9057846481318317
0.50, 0.9055502813571609
0.51, 0.9055502813571609
0.52, 0.9053140077794479
0.53, 0.9063655799872032
0.54, 0.9071853971691546
0.55, 0.9088389695646469
0.56, 0.9088389695646469
0.57, 0.9049188640973631
0.58, 0.9009663733110342
0.59, 0.8983130939836245
0.40, 0.9125694642230273


In [10]:
pred = xgb_pred
test_label['black_flag'] = (pred.reshape((-1))>best_threshold).astype('int')